In [ ]:
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import yaml
from pathlib import Path
from sklearn.preprocessing import LabelEncoder

# Import custom modules
from src.metrics import rmse
from src.features.preprocess import Preprocessor
from src.config.preprocess import PreprocessorConfig

# Set plotting style
plt.style.use("default")
sns.set_palette("husl")

In [ ]:
# 車両価格予測モデルの比較レポート

## 概要

本レポートでは、車両価格予測において、ベースライン手法と提案手法（Target Encoding + ハイパーパラメータ最適化）の性能を比較し、各手法の特徴と改善点について分析します。

### 評価データセット
- **訓練データ**: `projectA_vehicle_train.csv`
- **検証データ**: `projectA_vehicle_val.csv`  
- **テストデータ**: `projectA_vehicle_test.csv`

### 評価指標
- **RMSE (Root Mean Squared Error)**: 予測値と実際値の二乗平均平方根誤差

In [ ]:
# データ読み込み
dataset_path = Path("../dataset/")

# 不要な列を定義
unnecessary_columns = ["posting_date", "id"]

# Polars形式でデータ読み込み（提案手法用）
train_df_pl = pl.read_csv(dataset_path / "projectA_vehicle_train.csv").drop(
    unnecessary_columns
)
val_df_pl = pl.read_csv(dataset_path / "projectA_vehicle_val.csv").drop(
    unnecessary_columns
)
test_df_pl = pl.read_csv(dataset_path / "projectA_vehicle_test.csv").drop(
    unnecessary_columns
)

# Pandas形式でデータ読み込み（ベースライン手法用）
usecols = [
    "price",
    "year",
    "manufacturer",
    "condition",
    "cylinders",
    "fuel",
    "odometer",
    "transmission",
    "drive",
    "type",
    "paint_color",
]

train_df_pd = pd.read_csv(dataset_path / "projectA_vehicle_train.csv", usecols=usecols)
val_df_pd = pd.read_csv(dataset_path / "projectA_vehicle_val.csv", usecols=usecols)
test_df_pd = pd.read_csv(dataset_path / "projectA_vehicle_test.csv", usecols=usecols)

print("データセットの形状:")
print(f"訓練データ: {train_df_pd.shape}")
print(f"検証データ: {val_df_pd.shape}")
print(f"テストデータ: {test_df_pd.shape}")

# 基本統計量の表示
print("\n価格の基本統計量（訓練データ）:")
print(train_df_pd["price"].describe())

## 1. ベースライン手法

### 手法概要
ベースライン手法では、以下の特徴を持つシンプルなアプローチを採用：

1. **前処理**: 基本的な Label Encoding のみ
2. **モデル**: LightGBM with デフォルトパラメータ
3. **特徴量**: 基本的なカテゴリカル変数とそのまま数値変数を使用

### 主な問題点
- **単純な Label Encoding**: カテゴリの順序関係が価格予測に適さない可能性
- **ハイパーパラメータ未調整**: デフォルト設定による最適化不足
- **外れ値処理の不十分**: 極端な価格データの影響
- **特徴量エンジニアリングの不足**: カテゴリ間の関係性を捉えきれない

In [ ]:
# ベースライン手法の実装
def train_baseline_model(train_df, val_df, test_df):
    """
    ベースライン手法：Label Encoding + デフォルトLightGBM
    """
    # データのコピー
    train_processed = train_df.copy()
    val_processed = val_df.copy()
    test_processed = test_df.copy()

    # カテゴリカル変数の特定
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    categorical_columns = []
    for col in train_df.columns:
        if col != "price" and train_df[col].dtype not in numerics:
            categorical_columns.append(col)

    print("カテゴリカル変数:", categorical_columns)

    # Label Encoding
    for col in categorical_columns:
        le = LabelEncoder()
        le.fit(train_df[col].astype(str).values)
        train_processed[col] = le.transform(train_df[col].astype(str).values)
        val_processed[col] = le.transform(val_df[col].astype(str).values)
        test_processed[col] = le.transform(test_df[col].astype(str).values)

    # 外れ値フィルタリング（価格の上下限設定）
    train_filtered = train_processed[
        (train_processed["price"] > 1000) & (train_processed["price"] < 40000)
    ]

    # LightGBMデータセット作成
    train_set = lgb.Dataset(
        train_filtered.drop(columns="price"), train_filtered["price"]
    )
    val_set = lgb.Dataset(
        val_processed.drop(columns="price"), val_processed["price"], reference=train_set
    )

    # ベースラインパラメータ（デフォルト設定）
    baseline_params = {
        "boosting_type": "gbdt",
        "objective": "regression",
        "metric": "rmse",
        "num_leaves": 31,
        "learning_rate": 0.1,
        "seed": 42,
        "verbose": -1,
    }

    # モデル訓練
    model = lgb.train(
        baseline_params,
        train_set,
        num_boost_round=1000,
        valid_sets=[val_set],
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)],
    )

    return model, train_filtered, val_processed, test_processed


# ベースライン手法の実行
print("=== ベースライン手法の訓練 ===")
baseline_model, train_baseline, val_baseline, test_baseline = train_baseline_model(
    train_df_pd, val_df_pd, test_df_pd
)

In [ ]:
# ベースライン手法の評価
def evaluate_model(model, train_df, val_df, test_df, method_name):
    """
    モデルの評価を行い、RMSE スコアを計算
    """
    results = {}

    # 訓練データでの評価
    train_pred = model.predict(train_df.drop(columns="price"))
    train_rmse = rmse(train_df["price"].values, train_pred)
    results["train_rmse"] = train_rmse

    # 検証データでの評価
    val_pred = model.predict(val_df.drop(columns="price"))
    val_rmse = rmse(val_df["price"].values, val_pred)
    results["val_rmse"] = val_rmse

    # テストデータでの評価
    test_pred = model.predict(test_df.drop(columns="price"))
    test_rmse = rmse(test_df["price"].values, test_pred)
    results["test_rmse"] = test_rmse

    print(f"=== {method_name} 評価結果 ===")
    print(f"訓練データ RMSE: ${train_rmse:,.2f}")
    print(f"検証データ RMSE: ${val_rmse:,.2f}")
    print(f"テストデータ RMSE: ${test_rmse:,.2f}")

    return results, {"train": train_pred, "val": val_pred, "test": test_pred}


# ベースライン手法の評価
baseline_results, baseline_predictions = evaluate_model(
    baseline_model, train_baseline, val_baseline, test_baseline, "ベースライン手法"
)

## 2. 提案手法（Target Encoding + ハイパーパラメータ最適化）

### 手法概要
提案手法では、ベースライン手法の問題点を以下の方法で解決：

1. **高度な特徴量エンジニアリング**: Target Encoding による統計的特徴量生成
2. **過学習抑制**: Smoothing と Noise Level による正則化
3. **ハイパーパラメータ最適化**: Optuna による自動最適化
4. **複合エンコーディング**: Label Encoding + Target Encoding + Grouping の組み合わせ

### Target Encoding の利点
- **統計的意味**: カテゴリごとの目的変数（価格）の統計量を特徴量として活用
- **過学習抑制**: Smoothing パラメータにより少数サンプルカテゴリの影響を調整
- **ノイズ付加**: 訓練データへの過度な適合を防ぐためのランダムノイズ
- **最小サンプル数**: 統計的信頼性を確保するための閾値設定

### 最適化されたパラメータ
以下のパラメータがOptuna により最適化されています：

In [ ]:
# 最適化されたパラメータの読み込み
params_path = Path("../params/")

# LightGBM パラメータの読み込み
with open(params_path / "best_lgb_params_reg.yaml", "r") as f:
    best_lgb_params = yaml.safe_load(f)

# 前処理設定の読み込み
best_preprocessor_config = PreprocessorConfig.from_yaml(
    params_path / "best_preprocessor_config_reg.yaml"
)

print("=== 最適化されたLightGBMパラメータ ===")
for key, value in best_lgb_params.items():
    print(f"{key}: {value}")

print("\n=== Target Encoding パラメータ ===")
target_config = best_preprocessor_config.condition_encoder_config.target_encoder_config
print(f"Smoothing: {target_config.smoothing:.6f}")
print(f"Min Samples Leaf: {target_config.min_samples_leaf}")
print(f"Noise Level: {target_config.noise_level:.6f}")

print(f"\n=== 価格フィルタリング設定 ===")
print(f"価格下限: ${best_preprocessor_config.price_lower_bound:,.0f}")
print(f"価格上限: ${best_preprocessor_config.price_upper_bound:,.0f}")

In [ ]:
# 提案手法の実装
def train_proposed_model(train_df, val_df, test_df, preprocessor_config, lgb_params):
    """
    提案手法：Target Encoding + 最適化ハイパーパラメータ
    """
    print("=== 提案手法：前処理の実行 ===")
    # 前処理の実行
    preprocessor = Preprocessor(**preprocessor_config.to_dict())
    train_processed, val_processed, test_processed = preprocessor.run(
        train_df, val_df, test_df
    )

    print(f"前処理後の特徴量数: {train_processed.select(pl.exclude('price')).shape[1]}")

    # LightGBMデータセット作成
    train_set = lgb.Dataset(
        train_processed.drop("price").to_pandas(), train_processed["price"].to_pandas()
    )
    val_set = lgb.Dataset(
        val_processed.drop("price").to_pandas(),
        val_processed["price"].to_pandas(),
        reference=train_set,
    )

    print("=== 提案手法：モデル訓練 ===")
    # モデル訓練
    model = lgb.train(
        lgb_params,
        train_set,
        num_boost_round=best_lgb_params["n_estimators"],
        valid_sets=[val_set],
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)],
    )

    return model, train_processed, val_processed, test_processed


# 提案手法の実行
proposed_model, train_proposed, val_proposed, test_proposed = train_proposed_model(
    train_df_pl, val_df_pl, test_df_pl, best_preprocessor_config, best_lgb_params
)

In [ ]:
# 提案手法の評価（Polars対応版）
def evaluate_model_polars(model, train_df, val_df, test_df, method_name):
    """
    Polarsデータフレーム用のモデル評価関数
    """
    results = {}

    # 訓練データでの評価
    train_pred = model.predict(train_df.drop("price").to_pandas())
    train_rmse = rmse(train_df["price"].to_numpy(), train_pred)
    results["train_rmse"] = train_rmse

    # 検証データでの評価
    val_pred = model.predict(val_df.drop("price").to_pandas())
    val_rmse = rmse(val_df["price"].to_numpy(), val_pred)
    results["val_rmse"] = val_rmse

    # テストデータでの評価
    test_pred = model.predict(test_df.drop("price").to_pandas())
    test_rmse = rmse(test_df["price"].to_numpy(), test_pred)
    results["test_rmse"] = test_rmse

    print(f"=== {method_name} 評価結果 ===")
    print(f"訓練データ RMSE: ${train_rmse:,.2f}")
    print(f"検証データ RMSE: ${val_rmse:,.2f}")
    print(f"テストデータ RMSE: ${test_rmse:,.2f}")

    return results, {"train": train_pred, "val": val_pred, "test": test_pred}


# 提案手法の評価
proposed_results, proposed_predictions = evaluate_model_polars(
    proposed_model, train_proposed, val_proposed, test_proposed, "提案手法"
)

## 3. 手法比較と結果分析

### 3.1 RMSE スコア比較

In [ ]:
# 結果比較の可視化
def create_comparison_plots():
    """
    ベースライン手法と提案手法の比較プロット作成
    """
    # 結果をまとめる
    comparison_df = pd.DataFrame(
        {
            "Dataset": ["Train", "Validation", "Test"],
            "Baseline": [
                baseline_results["train_rmse"],
                baseline_results["val_rmse"],
                baseline_results["test_rmse"],
            ],
            "Proposed": [
                proposed_results["train_rmse"],
                proposed_results["val_rmse"],
                proposed_results["test_rmse"],
            ],
        }
    )

    # 改善率の計算
    comparison_df["Improvement (%)"] = (
        (comparison_df["Baseline"] - comparison_df["Proposed"])
        / comparison_df["Baseline"]
        * 100
    )

    # プロットの作成
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # RMSE比較
    x = np.arange(len(comparison_df))
    width = 0.35

    axes[0].bar(
        x - width / 2,
        comparison_df["Baseline"],
        width,
        label="Baseline",
        alpha=0.8,
        color="lightcoral",
    )
    axes[0].bar(
        x + width / 2,
        comparison_df["Proposed"],
        width,
        label="Proposed",
        alpha=0.8,
        color="skyblue",
    )

    axes[0].set_xlabel("Dataset")
    axes[0].set_ylabel("RMSE ($)")
    axes[0].set_title("RMSE Comparison: Baseline vs Proposed Method")
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(comparison_df["Dataset"])
    axes[0].legend()
    axes[0].grid(axis="y", alpha=0.3)

    # 値をバーの上に表示
    for i, (baseline, proposed) in enumerate(
        zip(comparison_df["Baseline"], comparison_df["Proposed"])
    ):
        axes[0].text(
            i - width / 2,
            baseline + 200,
            f"${baseline:,.0f}",
            ha="center",
            va="bottom",
            fontsize=10,
        )
        axes[0].text(
            i + width / 2,
            proposed + 200,
            f"${proposed:,.0f}",
            ha="center",
            va="bottom",
            fontsize=10,
        )

    # 改善率のプロット
    colors = ["green" if x > 0 else "red" for x in comparison_df["Improvement (%)"]]
    bars = axes[1].bar(
        comparison_df["Dataset"],
        comparison_df["Improvement (%)"],
        color=colors,
        alpha=0.7,
    )

    axes[1].set_xlabel("Dataset")
    axes[1].set_ylabel("Improvement (%)")
    axes[1].set_title("Performance Improvement (%)")
    axes[1].grid(axis="y", alpha=0.3)
    axes[1].axhline(y=0, color="black", linestyle="-", alpha=0.3)

    # 改善率の値を表示
    for bar, improvement in zip(bars, comparison_df["Improvement (%)"]):
        height = bar.get_height()
        axes[1].text(
            bar.get_x() + bar.get_width() / 2.0,
            height + (0.5 if height > 0 else -1),
            f"{improvement:.1f}%",
            ha="center",
            va="bottom" if height > 0 else "top",
        )

    plt.tight_layout()
    plt.show()

    return comparison_df


# 比較表とプロットの作成
print("=== 手法比較結果 ===")
comparison_results = create_comparison_plots()
print("\n詳細比較表:")
print(comparison_results.round(2))

### 3.2 特徴量重要度分析

In [ ]:
# 特徴量重要度の比較
def compare_feature_importance():
    """
    ベースライン手法と提案手法の特徴量重要度を比較
    """
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))

    # ベースライン手法の特徴量重要度
    baseline_importance = baseline_model.feature_importance(importance_type="gain")
    baseline_features = train_baseline.drop(columns="price").columns.tolist()

    baseline_df = (
        pd.DataFrame({"feature": baseline_features, "importance": baseline_importance})
        .sort_values("importance", ascending=True)
        .tail(10)
    )

    axes[0].barh(
        baseline_df["feature"], baseline_df["importance"], color="lightcoral", alpha=0.8
    )
    axes[0].set_title("Baseline Method: Top 10 Feature Importance", fontsize=14)
    axes[0].set_xlabel("Importance (Gain)")
    axes[0].grid(axis="x", alpha=0.3)

    # 提案手法の特徴量重要度
    proposed_importance = proposed_model.feature_importance(importance_type="gain")
    proposed_features = train_proposed.drop("price").columns

    proposed_df = (
        pd.DataFrame({"feature": proposed_features, "importance": proposed_importance})
        .sort_values("importance", ascending=True)
        .tail(15)
    )

    axes[1].barh(
        proposed_df["feature"], proposed_df["importance"], color="skyblue", alpha=0.8
    )
    axes[1].set_title("Proposed Method: Top 15 Feature Importance", fontsize=14)
    axes[1].set_xlabel("Importance (Gain)")
    axes[1].grid(axis="x", alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("=== ベースライン手法 トップ10 特徴量 ===")
    for idx, row in baseline_df.iterrows():
        print(f"{row['feature']}: {row['importance']:.0f}")

    print("\n=== 提案手法 トップ15 特徴量 ===")
    for idx, row in proposed_df.iterrows():
        print(f"{row['feature']}: {row['importance']:.0f}")

    return baseline_df, proposed_df


baseline_importance_df, proposed_importance_df = compare_feature_importance()

### 3.3 予測精度の詳細分析

In [ ]:
# 予測精度の詳細分析
def analyze_prediction_quality():
    """
    両手法の予測精度を詳細分析
    """
    # 検証データでの分析
    val_actual = val_baseline["price"].values
    baseline_pred = baseline_predictions["val"]
    proposed_pred = proposed_predictions["val"]

    # 残差の計算
    baseline_residuals = val_actual - baseline_pred
    proposed_residuals = val_proposed["price"].to_numpy() - proposed_pred

    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 実際値 vs 予測値の散布図
    axes[0, 0].scatter(
        val_actual, baseline_pred, alpha=0.5, s=20, color="lightcoral", label="Baseline"
    )
    axes[0, 0].scatter(
        val_proposed["price"].to_numpy(),
        proposed_pred,
        alpha=0.5,
        s=20,
        color="skyblue",
        label="Proposed",
    )
    max_price = max(val_actual.max(), baseline_pred.max(), proposed_pred.max())
    axes[0, 0].plot(
        [0, max_price], [0, max_price], "r--", alpha=0.8, label="Perfect Prediction"
    )
    axes[0, 0].set_xlabel("Actual Price ($)")
    axes[0, 0].set_ylabel("Predicted Price ($)")
    axes[0, 0].set_title("Actual vs Predicted Price (Validation Set)")
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # 残差の分布比較
    axes[0, 1].hist(
        baseline_residuals,
        bins=50,
        alpha=0.7,
        color="lightcoral",
        label=f"Baseline (std={np.std(baseline_residuals):.0f})",
        density=True,
    )
    axes[0, 1].hist(
        proposed_residuals,
        bins=50,
        alpha=0.7,
        color="skyblue",
        label=f"Proposed (std={np.std(proposed_residuals):.0f})",
        density=True,
    )
    axes[0, 1].axvline(x=0, color="red", linestyle="--", alpha=0.8)
    axes[0, 1].set_xlabel("Residuals (Actual - Predicted)")
    axes[0, 1].set_ylabel("Density")
    axes[0, 1].set_title("Distribution of Residuals")
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # 価格帯別の誤差分析
    price_bins = pd.cut(
        val_actual, bins=5, labels=["Very Low", "Low", "Medium", "High", "Very High"]
    )
    baseline_errors_by_price = (
        pd.DataFrame({"price_bin": price_bins, "abs_error": np.abs(baseline_residuals)})
        .groupby("price_bin")["abs_error"]
        .mean()
    )

    proposed_errors_by_price = (
        pd.DataFrame({"price_bin": price_bins, "abs_error": np.abs(proposed_residuals)})
        .groupby("price_bin")["abs_error"]
        .mean()
    )

    x = np.arange(len(baseline_errors_by_price))
    width = 0.35
    axes[1, 0].bar(
        x - width / 2,
        baseline_errors_by_price.values,
        width,
        label="Baseline",
        color="lightcoral",
        alpha=0.8,
    )
    axes[1, 0].bar(
        x + width / 2,
        proposed_errors_by_price.values,
        width,
        label="Proposed",
        color="skyblue",
        alpha=0.8,
    )
    axes[1, 0].set_xlabel("Price Range")
    axes[1, 0].set_ylabel("Mean Absolute Error ($)")
    axes[1, 0].set_title("Mean Absolute Error by Price Range")
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels(baseline_errors_by_price.index, rotation=45)
    axes[1, 0].legend()
    axes[1, 0].grid(axis="y", alpha=0.3)

    # 相対誤差の比較
    baseline_rel_error = np.abs(baseline_residuals) / val_actual
    proposed_rel_error = np.abs(proposed_residuals) / val_proposed["price"].to_numpy()

    axes[1, 1].boxplot(
        [baseline_rel_error, proposed_rel_error], labels=["Baseline", "Proposed"]
    )
    axes[1, 1].set_ylabel("Relative Error (|residual| / actual_price)")
    axes[1, 1].set_title("Relative Error Distribution")
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # 統計サマリーの表示
    print("=== 予測精度統計サマリー（検証データ） ===")
    print(f"ベースライン手法:")
    print(f"  - MAE: ${np.mean(np.abs(baseline_residuals)):,.2f}")
    print(
        f"  - MAPE: {np.mean(baseline_rel_error):.3f} ({np.mean(baseline_rel_error) * 100:.1f}%)"
    )
    print(f"  - 残差標準偏差: ${np.std(baseline_residuals):,.2f}")

    print(f"\n提案手法:")
    print(f"  - MAE: ${np.mean(np.abs(proposed_residuals)):,.2f}")
    print(
        f"  - MAPE: {np.mean(proposed_rel_error):.3f} ({np.mean(proposed_rel_error) * 100:.1f}%)"
    )
    print(f"  - 残差標準偏差: ${np.std(proposed_residuals):,.2f}")


analyze_prediction_quality()

## 4. 結論と考察

### 4.1 主要な改善点

提案手法（Target Encoding + ハイパーパラメータ最適化）により、以下の改善が確認されました：

1. **予測精度の向上**: 検証データでのRMSE改善
2. **特徴量の有効活用**: Target Encodingにより統計的に意味のある特徴量を生成
3. **過学習の抑制**: Smoothing、Noise Level、min_samples_leafによる正則化効果
4. **ハイパーパラメータ最適化**: Optunaによる体系的な最適化

### 4.2 Target Encodingの効果

Target Encodingにより以下の利点が得られました：
- **統計的特徴量**: カテゴリごとの目的変数統計量を直接利用
- **高次元カテゴリ対応**: 多数のカテゴリを持つ変数（manufacturer, stateなど）の効果的処理
- **過学習防止**: 適切な正則化パラメータによる汎化性能向上

### 4.3 今後の改善可能性

さらなる性能向上のための方向性：
1. **外部データ統合**: 経済指標、地域情報など
2. **アンサンブル手法**: 複数モデルの組み合わせ
3. **深層学習**: ニューラルネットワークベースの手法
4. **特徴量選択**: より精密な特徴量選択手法

In [ ]:
# 最終サマリー表の作成
def create_final_summary():
    """
    両手法の最終的な比較サマリーを作成
    """
    summary_data = {
        "項目": [
            "RMSE (訓練)",
            "RMSE (検証)",
            "RMSE (テスト)",
            "特徴量数",
            "前処理手法",
            "ハイパーパラメータ",
            "過学習対策",
        ],
        "ベースライン手法": [
            f"${baseline_results['train_rmse']:,.0f}",
            f"${baseline_results['val_rmse']:,.0f}",
            f"${baseline_results['test_rmse']:,.0f}",
            f"{len(train_baseline.columns) - 1}個",
            "Label Encoding",
            "デフォルト設定",
            "簡易的フィルタリング",
        ],
        "提案手法": [
            f"${proposed_results['train_rmse']:,.0f}",
            f"${proposed_results['val_rmse']:,.0f}",
            f"${proposed_results['test_rmse']:,.0f}",
            f"{len(train_proposed.columns) - 1}個",
            "Target Encoding + Label Encoding",
            "Optuna最適化",
            "Smoothing + Noise + 統計的閾値",
        ],
        "改善率": [
            f"{((baseline_results['train_rmse'] - proposed_results['train_rmse']) / baseline_results['train_rmse'] * 100):+.1f}%",
            f"{((baseline_results['val_rmse'] - proposed_results['val_rmse']) / baseline_results['val_rmse'] * 100):+.1f}%",
            f"{((baseline_results['test_rmse'] - proposed_results['test_rmse']) / baseline_results['test_rmse'] * 100):+.1f}%",
            f"+{len(train_proposed.columns) - len(train_baseline.columns)}個",
            "統計的手法採用",
            "自動最適化",
            "多層的正則化",
        ],
    }

    summary_df = pd.DataFrame(summary_data)
    print("=== 最終比較サマリー ===")
    print(summary_df.to_string(index=False))

    return summary_df


final_summary = create_final_summary()

print("\n" + "=" * 80)
print("レポート作成完了!")
print("=" * 80)